In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('../data/student_performance_data.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2392 entries, 0 to 2391
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   StudentID          2392 non-null   int64  
 1   Age                2392 non-null   int64  
 2   Gender             2392 non-null   int64  
 3   Ethnicity          2392 non-null   int64  
 4   ParentalEducation  2392 non-null   int64  
 5   StudyTimeWeekly    2392 non-null   float64
 6   Absences           2392 non-null   int64  
 7   Tutoring           2392 non-null   int64  
 8   ParentalSupport    2392 non-null   int64  
 9   Extracurricular    2392 non-null   int64  
 10  Sports             2392 non-null   int64  
 11  Music              2392 non-null   int64  
 12  Volunteering       2392 non-null   int64  
 13  GPA                2392 non-null   float64
 14  GradeClass         2392 non-null   float64
dtypes: float64(3), int64(12)
memory usage: 280.4 KB


In [4]:
def tipifica_variable(df, umbral_categoria=10, umbral_continua=30.0, motrar_card=False):
    """
    Función para tipificar variables como binaria, categórica, numérica continua y numérica discreta.
    
    Params:
        df (pd.DataFrame): DataFrame de pandas.
        umbral_categoria (int): Valor entero que define el umbral de la cardinalidad para variables categóricas.
        umbral_continua (float): Valor flotante que define el umbral de la cardinalidad para variables numéricas continuas.
        motrar_card (bool): Si es True, incluye la cardinalidad y el porcentaje de cardinalidad. False por defecto. 
    
    Returns:
        DataFrame con las columnas (variables), la tipificación sugerida de cada una.     
        y el tipo real detectado por pandas. Si `motrar_card` es True, también incluye las columnas 
        "CARD" (cardinalidad absoluta) y "%_CARD" (porcentaje de cardinalidad relativa).
    """
    
    # Validación del DataFrame
    if not isinstance(df, pd.DataFrame):
        raise ValueError("El argumento 'df' debe ser un DataFrame de pandas válido.")
    
    # Validación de los umbrales
    if not isinstance(umbral_categoria, int) or umbral_categoria <= 0:
        raise ValueError("El 'umbral_categoria' debe ser un número entero mayor que 0.")
    
    if not isinstance(umbral_continua, (float, int)) or umbral_continua <= 0:
        raise ValueError("El 'umbral_continua' debe ser un número float mayor que 0.")
    
    # DataFrame inicial con cardinalidad y tipificación sugerida
    df_card = pd.DataFrame({
        "CARD": df.nunique(),
        "%_CARD": round((df.nunique() / len(df) * 100),2),
        "tipo_sugerido": "",
        "tipo_real": df.dtypes.astype(str)
    })
    
    # Tipo Binaria
    df_card.loc[df_card["CARD"] == 2, "tipo_sugerido"] = "Binaria"
    
    # Tipo Categórica
    df_card.loc[(df_card["CARD"] < umbral_categoria) & (df_card["tipo_sugerido"] == ""), "tipo_sugerido"] = "Categórica"
    
    # Tipo Numérica Continua
    df_card.loc[(df_card["CARD"] >= umbral_categoria) & (df_card["%_CARD"] >= umbral_continua), "tipo_sugerido"] = "Numerica Continua"
    
    # Tipo Numérica Discreta
    df_card.loc[(df_card["CARD"] >= umbral_categoria) & (df_card["%_CARD"] < umbral_continua), "tipo_sugerido"] = "Numerica Discreta"

    # Selección y renombrado de columnas
    df_card = df_card.reset_index().rename(columns={"index": "nombre_variable"})
    
    if motrar_card == False:
        return df_card[["nombre_variable", "tipo_sugerido", "tipo_real"]]
    else:
        return df_card[["nombre_variable", "CARD", "%_CARD", "tipo_sugerido", "tipo_real"]]


In [5]:
# Visualizo DF
df.head(5)

,StudentID,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA,GradeClass
0,1001,17,1,0,2,19.833723,7,1,2,0,0,1,0,2.929196,2.0
1,1002,18,0,0,1,15.408756,0,0,1,0,0,0,0,3.042915,1.0
2,1003,15,0,2,3,4.210570,26,0,2,0,0,0,0,0.112602,4.0
3,1004,17,1,0,3,10.028829,14,0,3,1,0,0,0,2.054218,3.0
4,1005,17,1,0,2,4.672495,17,1,3,0,0,0,0,1.288061,4.0


### Usos correctos

In [ ]:
# Ejemplo correcto de uso, con valores por defecto: df, umbral_categoria=10, umbral_continua=30.0, motrar_card=False
tipifica_variable(df) 

,nombre_variable,tipo_sugerido,tipo_real
0,StudentID,Numerica Continua,int64
1,Age,Categórica,int64
2,Gender,Binaria,int64
3,Ethnicity,Categórica,int64
4,ParentalEducation,Categórica,int64
5,StudyTimeWeekly,Numerica Continua,float64
6,Absences,Numerica Discreta,int64
7,Tutoring,Binaria,int64
8,ParentalSupport,Categórica,int64
9,Extracurricular,Binaria,int64


In [ ]:
# Ejemplo correcto de uso, con otros valores correctos y mostrando cardinalidad
# df, umbral_categoria=5, umbral_continua=10.0, motrar_card=True

tipifica_variable(df, 5, 10, True)

,nombre_variable,CARD,%_CARD,tipo_sugerido,tipo_real
0,StudentID,2392,100.00,Numerica Continua,int64
1,Age,4,0.17,Categórica,int64
2,Gender,2,0.08,Binaria,int64
3,Ethnicity,4,0.17,Categórica,int64
4,ParentalEducation,5,0.21,Numerica Discreta,int64
5,StudyTimeWeekly,2392,100.00,Numerica Continua,float64
6,Absences,30,1.25,Numerica Discreta,int64
7,Tutoring,2,0.08,Binaria,int64
8,ParentalSupport,5,0.21,Numerica Discreta,int64
9,Extracurricular,2,0.08,Binaria,int64


### Errores

In [8]:
# Ejemplo de errores
tipifica_variable(df, "string", 5)

ValueError: El 'umbral_categoria' debe ser un número entero mayor que 0.

In [9]:
# Ejemplo de errores
tipifica_variable(df, 2.5 , 5)

ValueError: El 'umbral_categoria' debe ser un número entero mayor que 0.

In [10]:
# Ejemplo de errores
tipifica_variable(df, 2 , -5)

ValueError: El 'umbral_continua' debe ser un número float mayor que 0.